#### Model Selection

##### Product Sale

In [1]:
# Load the pickle file in another notebook
import pickle
with open('Pickle_Files/Customer_Loan.pkl', 'rb') as f:
     df_train_cl= pickle.load(f)


In [2]:
df_train_cl.head()

,Sex,Age,Tenure,Count_CA,Count_SA,Count_MF,Count_OVD,Count_CC,ActBal_CA,ActBal_SA,...,VolumeDebCash_Card,VolumeDebCashless_Card,VolumeDeb_PaymentOrder,TransactionsDeb,TransactionsDeb_CA,TransactionsDebCash_Card,TransactionsDebCashless_Card,TransactionsDeb_PaymentOrder,Sale_CL,Revenue_CL
0,1.0,38.0,165,1,0.0,0.0,0.0,0.0,6752.244643,0.000000,...,0.000000,0.000000,714.285714,1.0,1.0,0.0,0.0,1.0,0,0.000000
1,0.0,49.0,44,1,0.0,0.0,0.0,0.0,43.523214,0.000000,...,0.000000,0.000000,121.928571,6.0,6.0,0.0,0.0,1.0,0,0.000000
2,1.0,54.0,34,1,1.0,0.0,0.0,1.0,29.024286,14447.801429,...,357.142857,444.597500,2076.785714,48.0,38.0,1.0,26.0,11.0,0,0.000000
3,1.0,29.0,106,1,0.0,0.0,0.0,0.0,27.035714,0.000000,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0,0.000000
4,1.0,14.0,100,1,0.0,0.0,1.0,0.0,345.686071,0.000000,...,0.000000,111.235714,19.285714,5.0,5.0,0.0,4.0,1.0,1,8.130714


In [3]:
from sklearn.model_selection import train_test_split

X_cl = df_train_cl.copy()
X_cl.drop(['Revenue_CL','Sale_CL'],inplace=True,axis=1)

y_sale_cl = df_train_cl['Sale_CL']
#y_revenue_cl = df_train_cl['Revenue_CL']

X_train,X_test,y_train,y_test = train_test_split(X_cl,y_sale_cl,test_size=0.2,random_state=0)